In [26]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

import pathlib


# Load Data

In [27]:
data_folders = [
    pathlib.Path(f'./data/bicing/truncated/{y}') for y in [2022]
]

In [28]:
pandas_kwargs = {
    'index_col': 0,
}
df = pd.concat([pd.read_csv(file, **pandas_kwargs) for data_folder in data_folders for file in
                data_folder.glob('*/*.csv')]).drop_duplicates()

In [29]:
df = df.assign(
    datetime=lambda x: pd.to_datetime(x.datetime),
    percentage_docks_available=df.num_docks_available / (df.num_docks_available + df.num_bikes_available),
).drop(columns=['num_docks_available', 'num_bikes_available']).sort_values(['station_id', 'datetime'])

In [30]:
df

,station_id,datetime,year,month,day,hour,percentage_docks_available
726,1,2022-01-01 00:00:00,2022,1,1,0,0.608696
1,1,2022-01-01 00:00:00,2022,1,1,0,0.625000
2,1,2022-01-01 01:00:00,2022,1,1,1,0.681818
3,1,2022-01-01 02:00:00,2022,1,1,2,0.722826
4,1,2022-01-01 03:00:00,2022,1,1,3,0.737458
...,...,...,...,...,...,...,...
362884,532,2022-11-13 04:00:00,2022,11,13,4,0.000000
362885,532,2022-11-13 05:00:00,2022,11,13,5,0.000000
362886,532,2022-11-13 06:00:00,2022,11,13,6,0.000000
362887,532,2022-11-13 07:00:00,2022,11,13,7,0.000000


### Add station info

In [31]:
station_info = pd.read_csv('./data/bicing_info.csv')
df = pd.merge(left=df, right=station_info[['station_id', 'lat', 'lon', 'altitude', 'post_code']],
              on=['station_id'])
df.head()

,station_id,datetime,year,month,day,hour,percentage_docks_available,lat,lon,altitude,post_code
0,1,2022-01-01 00:00:00,2022,1,1,0,0.608696,41.397978,2.180107,16,8013
1,1,2022-01-01 00:00:00,2022,1,1,0,0.625000,41.397978,2.180107,16,8013
2,1,2022-01-01 01:00:00,2022,1,1,1,0.681818,41.397978,2.180107,16,8013
3,1,2022-01-01 02:00:00,2022,1,1,2,0.722826,41.397978,2.180107,16,8013
4,1,2022-01-01 03:00:00,2022,1,1,3,0.737458,41.397978,2.180107,16,8013


### climate

In [32]:
df_climate_ = pd.read_csv('./data/clima.csv', parse_dates=['time'])

df_climate = df_climate_.assign(
    year=df_climate_.time.dt.year,
    month=df_climate_.time.dt.month,
    day=df_climate_.time.dt.day,
    hour=df_climate_.time.dt.hour
)
df = pd.merge(left=df, right=df_climate.drop(columns=['time']), on=['hour', 'day', 'month', 'year'])

In [33]:
df

,station_id,datetime,year,month,day,hour,percentage_docks_available,lat,lon,altitude,post_code,temperature_2m,total_cloud_cover,total_precipitation,windspeed_10m
0,1,2022-01-01 00:00:00,2022,1,1,0,0.608696,41.397978,2.180107,16,8013,282.597015,0.339310,4.111789e-07,2.181416
1,1,2022-01-01 00:00:00,2022,1,1,0,0.625000,41.397978,2.180107,16,8013,282.597015,0.339310,4.111789e-07,2.181416
2,2,2022-01-01 00:00:00,2022,1,1,0,0.356322,41.395488,2.177198,17,8013,282.597015,0.339310,4.111789e-07,2.181416
3,2,2022-01-01 00:00:00,2022,1,1,0,0.364943,41.395488,2.177198,17,8013,282.597015,0.339310,4.111789e-07,2.181416
4,3,2022-01-01 00:00:00,2022,1,1,0,0.555556,41.394156,2.181331,11,8013,282.597015,0.339310,4.111789e-07,2.181416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419981,515,2022-01-13 11:00:00,2022,1,13,11,0.971429,41.435207,2.194800,19,8030,282.674469,0.007935,0.000000e+00,2.058891
4419982,516,2022-01-13 11:00:00,2022,1,13,11,0.250000,41.435460,2.200157,15,8030,282.674469,0.007935,0.000000e+00,2.058891
4419983,517,2022-01-13 11:00:00,2022,1,13,11,0.300000,41.462095,2.178959,44,8033,282.674469,0.007935,0.000000e+00,2.058891
4419984,518,2022-01-13 11:00:00,2022,1,13,11,1.000000,41.424689,2.157049,112,8032,282.674469,0.007935,0.000000e+00,2.058891


In [34]:
import numpy as np

# merge with previous hours
df = df.sort_values(['station_id', 'datetime']).assign(
    day_of_week=lambda x: x.datetime.dt.day_of_week,
    is_weekend=lambda x: x.day_of_week >= 5,
    is_night=lambda x: np.bitwise_or(x.hour >= 20, x.hour <= 7),
    is_work_morning=lambda x: np.bitwise_and(x.hour >= 6, x.hour <= 10) & np.bitwise_not(x.is_weekend),
    is_summer=lambda x: x.month.between(6, 8),
    ctx_1=lambda x: x.percentage_docks_available.shift(1),
    ctx_2=lambda x: x.percentage_docks_available.shift(2),
    ctx_3=lambda x: x.percentage_docks_available.shift(3),
    ctx_4=lambda x: x.percentage_docks_available.shift(4),
    station_id_aux=lambda x: x.station_id.shift(4),
    altitude=lambda x: x.altitude.astype(int)
)

In [35]:
df = df.query('not ctx_1.isnull()') \
    .query('not ctx_2.isnull()') \
    .query('not ctx_3.isnull()') \
    .query('not ctx_4.isnull()') \
    .query('station_id == station_id_aux') \
    .drop(columns=['station_id_aux']) \
    .query('not percentage_docks_available.isnull()')


In [36]:
dfnans = df.isna().sum()
assert dfnans[dfnans > 0].empty

In [37]:
df.head()

,station_id,datetime,year,month,day,hour,percentage_docks_available,lat,lon,altitude,...,windspeed_10m,day_of_week,is_weekend,is_night,is_work_morning,is_summer,ctx_1,ctx_2,ctx_3,ctx_4
2020,1,2022-01-01 03:00:00,2022,1,1,3,0.737458,41.397978,2.180107,16,...,2.155734,5,True,True,False,False,0.722826,0.681818,0.625000,0.608696
2525,1,2022-01-01 04:00:00,2022,1,1,4,0.717391,41.397978,2.180107,16,...,2.059913,5,True,True,False,False,0.737458,0.722826,0.681818,0.625000
3030,1,2022-01-01 05:00:00,2022,1,1,5,0.717391,41.397978,2.180107,16,...,1.985322,5,True,True,False,False,0.717391,0.737458,0.722826,0.681818
3535,1,2022-01-01 06:00:00,2022,1,1,6,0.672101,41.397978,2.180107,16,...,1.993759,5,True,True,False,False,0.717391,0.717391,0.737458,0.722826
4040,1,2022-01-01 07:00:00,2022,1,1,7,0.639706,41.397978,2.180107,16,...,2.046561,5,True,True,False,False,0.672101,0.717391,0.717391,0.737458


# CV

In [38]:
from sklearn import model_selection, linear_model, ensemble, neighbors, preprocessing, metrics, pipeline, compose

In [39]:
# CrossValidation

scoring = 'neg_root_mean_squared_error'


def get_cv_scores(model, X: pd.DataFrame, y: pd.Series, verbose=0):
    if verbose > 0: display(f'{X.shape=}')
    if verbose > 0: display(f'{y.shape=}')

    return model_selection.cross_val_score(
        model, X, y,
        scoring=scoring,
        cv=model_selection.TimeSeriesSplit(5),
        verbose=verbose,
    )

In [40]:
pipe = pipeline.Pipeline(
    [
        ("transformer", compose.ColumnTransformer(
            [
                ('bool', preprocessing.OneHotEncoder(handle_unknown="ignore"), ["is_weekend", "is_summer", "is_night"]),
                ('std', preprocessing.StandardScaler(), [f"ctx_{ii + 1}" for ii in range(4)]),
                ('4bins', preprocessing.KBinsDiscretizer(n_bins=4, encode='onehot'), ["hour", "month"]),
                ('2bins', preprocessing.KBinsDiscretizer(n_bins=2, encode='onehot'), ["temperature_2m"]),
            ],
            remainder="drop")
         ),
        ("regressor", linear_model.LinearRegression())
    ]
)
pipe

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('bool',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['is_weekend', 'is_summer',
                                                   'is_night']),
                                                 ('std', StandardScaler(),
                                                  ['ctx_1', 'ctx_2', 'ctx_3',
                                                   'ctx_4']),
                                                 ('4bins',
                                                  KBinsDiscretizer(n_bins=4),
                                                  ['hour', 'month']),
                                                 ('2bins',
                                                  KBinsDiscretizer(n_bins=2),
                                                  ['temperature_2m'])])),
                ('regressor', LinearRegression())])

In [41]:
# final train
Y_COLUMN = 'percentage_docks_available'

In [42]:
get_cv_scores(pipe, df.drop(columns=[Y_COLUMN]), df[Y_COLUMN], verbose=3)

'X.shape=(4416884, 23)'

'y.shape=(4416884,)'

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


[CV] END ............................... score: (test=-0.125) total time=   4.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s
/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


...

[CV] END ............................... score: (test=-0.119) total time=   9.8s


/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


[CV] END ............................... score: (test=-0.100) total time=  10.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   37.7s finished


array([-0.12541932, -0.11604383, -0.09883015, -0.11936493, -0.09962058])

In [43]:
pipe.fit(df.drop(columns=[Y_COLUMN]), df[Y_COLUMN])

/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:209: FutureWarning: In version 1.3 onwards, subsample=2e5 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('bool',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['is_weekend', 'is_summer',
                                                   'is_night']),
                                                 ('std', StandardScaler(),
                                                  ['ctx_1', 'ctx_2', 'ctx_3',
                                                   'ctx_4']),
                                                 ('4bins',
                                                  KBinsDiscretizer(n_bins=4),
                                                  ['hour', 'month']),
                                                 ('2bins',
                                                  KBinsDiscretizer(n_bins=2),
                                                  ['temperature_2m'])])),
                ('regressor', LinearRegression())])

In [44]:
# load X_val
X_val = pd.read_csv('./data/validation/X_validation.csv', index_col="index")
X_val = pd.merge(left=X_val, right=station_info[['station_id', 'lat', 'lon', 'altitude', 'post_code']],
                 on=['station_id'])
X_val = pd.merge(left=X_val, right=df_climate.drop(columns=['time']), on=['hour', 'day', 'month', 'year'])

X_val = X_val.assign(
    year=2023,
    date=lambda x: pd.to_datetime(dict(year=x.year, month=x.month, day=x.day)),
    day_of_week=lambda x: x.date.dt.day_of_week,
    is_weekend=lambda x: x.day_of_week >= 5,
    is_night=lambda x: np.bitwise_or(x.hour >= 20, x.hour <= 7),
    is_work_morning=lambda x: np.bitwise_and(x.hour >= 6, x.hour <= 10) & np.bitwise_not(x.is_weekend),
    is_summer=lambda x: x.month.between(6, 8),
).rename(
    columns={
        'ctx-1': 'ctx_1',
        'ctx-2': 'ctx_2',
        'ctx-3': 'ctx_3',
        'ctx-4': 'ctx_4',
    }
)

X_val.head()

KeyError: 'year'

In [ ]:
n = 7
y_val = pd.DataFrame(
    pipe.predict(X_val),
    index=X_val.index,
    columns=[Y_COLUMN]
).assign(index=X_val.index)[['index', "percentage_docks_available"]]

display(y_val.head())

y_val.to_csv(f'./results/v2_{n}.csv', index=False)